In [70]:
poke.data = read.csv("~\\Pokemon.csv")

In [71]:
# First, we attempt to classify a pokemon as Legendary, given its stats

# Also, we want to change our data to disregard the name. We also get
# rid of the Pokedex number of each pokemon.

rownames(poke.data) = poke.data[,2]
poke.data = poke.data[,-1]
poke.data = poke.data[,-1]

In [75]:
# We use Logistic Regression first, using ALL data 
# I also will get rid of the type of the pokemon, as this likely has no
# correlation to whether or not the pokemon is legendary

poke.data.notype = poke.data[,-1]
poke.data.notype = poke.data.notype[,-1]



log.fit = glm(Legendary ~., data = poke.data.notype, family = binomial)
coef(log.fit)

(Intercept)        Total           HP       Attack      Defense      Sp..Atk 
-25.15171841   0.05614780  -0.02053387  -0.03769314  -0.01800682  -0.01799148 
     Sp..Def        Speed   Generation 
 -0.01115455           NA   0.45209198

In [85]:
# The coefficients seem wonky to me. Let's see how accurate it is

log.prob = predict(log.fit, poke.data.notype, type = "response")


# Which pokemon is the fit most certain is a legendary? Which is the fit
# most certain is not a legendary?

which.max(log.prob)
log.prob[which.max(log.prob)]

which.min(log.prob)
log.prob[which.min(log.prob)]

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"

MewtwoMega Mewtwo Y 
                165

MewtwoMega Mewtwo Y 
          0.9943796

Sunkern 
    207

Sunkern 
3.059812e-08

In [123]:
# Apparently Mega Mewtwo Y is 99.4% likely to be a legendary based 
# on its stats. Conversely, it is very certain that Sunkern is not
# a legendary

# We will check how accurate our model was on our train data, by seeing
# how many pokemon were correctly classified

log.pred = rep("False", nrow(poke.data.notype))
log.pred[log.prob >= 0.5] = "True"

table(log.pred, poke.data.notype$Legendary)
mean(log.pred == poke.data.notype$Legendary)

        
log.pred False True
   False   720   27
   True     15   38

[1] 0.9475

In [152]:
# Our model correctly classified ~ 95% of the pokemon as legendary or 
# non-legendary, based on its stats. Keep in mind, I could have severely
# overfit this data, as this is only the training error. In practice, the
# test error is much higher. We will explore this in a moment, using 
# cross validation

In [153]:
# Here is something interesting to note. Since 92% of pokemon are not
# not legendary, a model where we simply guess that each pokemon is 
# not legendary wouldn't be much worse than our current model.

In [116]:
sum(poke.data.notype$Legendary == "False") / nrow(poke.data.notype)

[1] 735

[1] 0.91875

In [151]:
# Which pokemon were incorrectly classified?

nonlegendary.wrong = rep(NA,26)
k = nrow(poke.data.notype)
j = 0
for(i in 1:k) {
    if(log.pred[i] == "False" && 
       poke.data.notype$Legendary[i] == "True") {
        nonlegendary.wrong[j] = rownames(poke.data.notype)[i]
        j = j + 1
    }
}


legendary.wrong = rep(NA,14)
k = nrow(poke.data.notype)
j = 0
for(i in 1:k) {
    if(log.pred[i] == "True" && 
       poke.data.notype$Legendary[i] == "False") {
        legendary.wrong[j] = rownames(poke.data.notype)[i]
        j = j + 1
    }
}

nonlegendary.wrong
legendary.wrong

[1] "Zapdos"                   "Moltres"                 
 [3] "Raikou"                   "Entei"                   
 [5] "Suicune"                  "Regirock"                
 [7] "Regice"                   "Registeel"               
 [9] "Latios"                   "Jirachi"                 
[11] "DeoxysNormal Forme"       "DeoxysAttack Forme"      
[13] "Uxie"                     "Mesprit"                 
[15] "Azelf"                    "Heatran"                 
[17] "ShayminLand Forme"        "Cobalion"                
[19] "Terrakion"                "TornadusIncarnate Forme" 
[21] "TornadusTherian Forme"    "ThundurusIncarnate Forme"
[23] "ThundurusTherian Forme"   "LandorusIncarnate Forme" 
[25] "LandorusTherian Forme"    "Volcanion"

[1] "TyranitarMega Tyranitar" "SceptileMega Sceptile"  
 [3] "GardevoirMega Gardevoir" "Slaking"                
 [5] "SalamenceMega Salamence" "MetagrossMega Metagross"
 [7] "GarchompMega Garchomp"   "Cresselia"              
 [9] "Hydreigon"               "KeldeoOrdinary Forme"   
[11] "KeldeoResolute Forme"    "MeloettaAria Forme"     
[13] "MeloettaPirouette Forme" "Goodra"

In [220]:
# Now we will see how good logistic Regression actually works, using
# cross validation to find out OOB test error

# We will use 10-fold validation

set.seed(1)
folds = sample(1:10, nrow(poke.data.notype), replace = TRUE)
cv.error = rep(0,10)
for(i in 1:10) {
    train.log.fit = glm(Legendary ~., 
                        data = poke.data.notype[i != folds,],
                        family = "binomial")
    train.log.prob = predict(train.log.fit,
                            poke.data.notype[i == folds,],
                            type = "response")
    train.log.pred = rep("False", length(train.log.prob))
    train.log.pred[train.log.prob >= 0.5] = "True"
    cv.error[i] = mean(train.log.pred != 
                       poke.data.notype$Legendary[i == folds])
}

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient fit may be misleading"Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = ifelse(type == :
"prediction from a rank-deficient 

In [221]:
mean(1 - cv.error)

[1] 0.9435743

In [206]:
# Our model doesn't preform terribly, correctly classifying about 95% 
# of pokemon correctly.

# Quickly, I will preform Linear Discriminant Analysis

library(MASS)

cv.error.lda = rep(0,10)
for(i in 1:10) {
    train.lda.fit = lda(Legendary ~., 
                        data = poke.data.notype[i != folds,])
    train.lda.pred = predict(train.lda.fit, 
                             poke.data.notype[i == folds,])$class
    cv.error.lda[i] = mean(train.lda.pred != 
                       poke.data.notype$Legendary[i == folds])
}

Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"Warning message in lda.default(x, grouping, ...):
"variables are collinear"

In [205]:
1 - mean(cv.error.lda)

[1] 0.9384174

In [208]:
# This is slightly better than just guessing that each pokemon is not
# legendary. Also, note that this has revealed a critical flaw in our
# analysis. Our predictors are collinear! Observe:

cor(poke.data.notype[,-9])

,Total,HP,Attack,Defense,Sp..Atk,Sp..Def,Speed,Generation
Total,1.00000000,0.61874835,0.73621065,0.61278743,0.74724986,0.71760947,0.57594266,0.04838402
HP,0.61874835,1.00000000,0.42238603,0.23962232,0.36237986,0.37871807,0.17595206,0.05868251
Attack,0.73621065,0.42238603,1.00000000,0.43868706,0.39636176,0.26398955,0.38123974,0.05145134
Defense,0.61278743,0.23962232,0.43868706,1.00000000,0.22354861,0.51074659,0.01522660,0.04241857
Sp..Atk,0.74724986,0.36237986,0.39636176,0.22354861,1.00000000,0.50612142,0.47301788,0.03643683
Sp..Def,0.71760947,0.37871807,0.26398955,0.51074659,0.50612142,1.00000000,0.25913311,0.02848599
Speed,0.57594266,0.17595206,0.38123974,0.01522660,0.47301788,0.25913311,1.00000000,-0.02312106
Generation,0.04838402,0.05868251,0.05145134,0.04241857,0.03643683,0.02848599,-0.02312106,1.00000000


In [257]:
# There are very large correlations between some stats. For example,
# r = 0.5 between Sp.Def and Defence

# This has me wondering if perhaps a better measure would be to only
# look at the total of the stats, or maybe to disregard the total?

# Using only total stats:

folds = sample(1:10, nrow(poke.data.notype), rep = TRUE)
cv.error.logfit2 = rep(0,10)
for(i in 1:10) {
    log.fit2 = glm(Legendary ~ Total, data = poke.data.notype[i != folds,], family = "binomial")
    log.fit2.prob = predict(log.fit2, poke.data.notype[i == folds,], type = "response")
    log.fit2.pred = rep("False", length(log.fit2.prob))
    log.fit2.pred[log.fit2.prob >= 0.5] = "True"
    cv.error.logfit2[i] = mean(log.fit2.pred != poke.data.notype$Legendary[i == folds])
}


# Diregarding total stats:

folds = sample(1:10, nrow(poke.data.notype), rep = TRUE)
cv.error.logfit3 = rep(0,10)
for(i in 1:10) {
    log.fit3 = glm(Legendary ~ .-Total, data = poke.data.notype[i != folds,], family = "binomial")
    log.fit3.prob = predict(log.fit3, poke.data.notype[i == folds,], type = "response")
    log.fit3.pred = rep("False", length(log.fit3.prob))
    log.fit3.pred[log.fit3.prob >= 0.5] = "True"
    cv.error.logfit3[i] = mean(log.fit3.pred != poke.data.notype$Legendary[i == folds])
}


1 - mean(cv.error.logfit2)
1 - mean(cv.error.logfit3)

[1] 0.9338743

[1] 0.9415331

In [286]:
# Finally, we consider the k-nearest neighbour approach.

library(DMwR)


folds = sample(1:10, nrow(poke.data.notype), rep = TRUE)
cv.error.logfit4 = rep(0,10)
for(i in 1:10) {
    train.x = poke.data.notype[i != folds,]
    test.x = poke.data.notype[i == folds,]
    train.y = poke.data.notype$Legendary[i != folds]
    test.y = poke.data.notype$Legendary[i == folds]
    knn.fit = kNN(Legendary ~., train.x, test.x, norm = FALSE, k = 1)
    cv.error.logfit4[i] = mean(knn.fit != poke.data.notype$Legendary[i == folds])
}

In [285]:
1 - mean(cv.error.logfit4)

[1] 0.9564262

In [ ]:
## The k-nearest neighbours approach has the lowest test error rate. Cool!

In [287]:
fix(poke.data)